# Concha Tutorial
Follow this instructions to understand how to deal with Concha and its concepts:

* Concha Service
* Servers auxiliar mock Service
* Documents
* Tricks
* GET and POST Tricks
* TREAT Tricks and dependent clauses
* Mixing up it all

You should be executing this very notebook running `jupyter concha_tutorial.ipynb` in a console to follow the instructions to master Concha usage. Please run cells sequentially to do that.

In [1]:
# Please run first this cell to set up rest of executions
import requests

def post(uri, body):
    response = requests.post(url=uri, json=body)
    print('status={}'.format(response.status_code))
    print(response.text)

def get(uri):
    response = requests.get(url=uri)
    print('status={}'.format(response.status_code))
    print(response.text)

## Concha (v1) Service
Concha runs as a RESTful service you must execute in `concha/concha` directory running `python3 concha` in a console. It will offer an end-point in `localhost:5000`.

### Concha Resources
Concha deals with two resources: **documents** and **tricks**. Documents are the plain natural language sentences you tell Concha to interact with. Tricks are the "programs" you instruct Concha with to know how to deal with each document. Concha will select the right combination among all stored tricks to react to every document it receive.

In [2]:
# You should be able to verify concha service is 
# up and running if execution returns an empty list:
"""
status=200
[]
"""
get('http://localhost:5000/v1/documents')

status=200
[]



## Servers (v1) auxiliar mock Service
In order to have some predictable mock service to interact with, you can execute in `concha/mocks` directory the command `python3 servers`. It will offer an end point in `localhost:6000`.

### Servers Resources
Servers offers one single resource: **servers**. It simulates the management of a fake cluster of servers (a micro cloud with no real servers behind) you can query, start and stop.

In [3]:
# You should be able to verify servers service is 
# up and running if execution returns an empty JSON:
"""
status=200
{}
"""
get('http://localhost:6000/v1/servers')

status=200
{
  "srv1": {
    "description": {
      "kind": "cheap machine"
    }, 
    "load": 10, 
    "name": "srv1", 
    "p1": {
      "description": {
        "kind": "tetris"
      }, 
      "name": "p1"
    }, 
    "processes": {}
  }, 
  "srv7": {
    "description": {
      "kind": "cheap machine"
    }, 
    "load": 20, 
    "name": "srv7", 
    "p1": {
      "description": {
        "kind": "pacman"
      }, 
      "name": "p1"
    }, 
    "p2": {
      "description": {
        "kind": "pong"
      }, 
      "name": "p2"
    }, 
    "processes": {}
  }
}



### Server Methods
There's just a plain CRUD set of methods:
* **POST /v1/servers/server_name** A new server named server_name (if it doesn't exist yet) with whatever JSON description in the body.
* **GET /v1/servers** The named set of all servers with its description and fake load.
* **GET /v1/servers/server_name** The description and fake load of server_name server.
* **PUT /v1/servers/server_name** Modify server named server_name (if it already exists) with whatever new JSON description in the body.
* **DELETE /v1/servers/server_name** Delete server named server_name (if it already exists).

There are similar methods for the sub-resource `/v1/servers/server_name/processes/process_name`

Each process will consume 10 hypothetical units of the server where it runs.

In [4]:
# Let's create a new srv1 server
post(
    'http://localhost:6000/v1/servers/srv1',
    {"kind": "cheap machine"}
)

# And verify it is there
get('http://localhost:6000/v1/servers')

status=409
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>409 Conflict</title>
<h1>Conflict</h1>
<p>A conflict happened while processing the request.  The resource might have been modified while the request was being processed.</p>

status=200
{
  "srv1": {
    "description": {
      "kind": "cheap machine"
    }, 
    "load": 10, 
    "name": "srv1", 
    "p1": {
      "description": {
        "kind": "tetris"
      }, 
      "name": "p1"
    }, 
    "processes": {}
  }, 
  "srv7": {
    "description": {
      "kind": "cheap machine"
    }, 
    "load": 20, 
    "name": "srv7", 
    "p1": {
      "description": {
        "kind": "pacman"
      }, 
      "name": "p1"
    }, 
    "p2": {
      "description": {
        "kind": "pong"
      }, 
      "name": "p2"
    }, 
    "processes": {}
  }
}



## Documents
Documents resources are stored in memory (no persistence yet) in a sequential way (append only).

### Documents Methods
Methods are as few as possible.
* **POST /v1/documents:analyzeSyntax** Fancy method to get the syntactic analysis of the given text. This is an instrumental function to help to think in the development of new _tricks_ (more on this later).

_Request body:_ The text you want Concha deal with
```javascript
{
	"text": "mi mamá me mima"
}
```

_Response body:_ A JSON structure with the parsing of the text according to the [Universal Dependencies annotation guidelines](http://universaldependencies.org). It's weird, but legible, and you can consider it as the internal Concha's assembler. You just need to know _ID_ is sequential order of the word in the text (starting by 1), _FORM_ is the word itself, and the _key_ of each compound object (i.e. _"nsubj"_ = nominal subject) is the [syntactic relationship](http://universaldependencies.org/u/dep/index.html) to the parent key.
```javascript
{
    "ROOT": {
        "FEATS": "Mood=Ind|Number=Sing|Person=1|Tense=Past|VerbForm=Fin|fPOS=VERB++",
        "FORM": "mima",
        "ID": "4",
        "UPOSTAG": "VERB",
        "iobj": {
            "FEATS": "Case=Acc,Dat|Number=Sing|Person=1|PrepCase=Npr|PronType=Prs|Reflex=Yes|fPOS=PRON++",
            "FORM": "me",
            "ID": "3",
            "UPOSTAG": "PRON"
        },
        "nsubj": {
            "FEATS": "Gender=Fem|Number=Sing|fPOS=NOUN++",
            "FORM": "mamá",
            "ID": "2",
            "UPOSTAG": "NOUN",
            "det": {
                "FEATS": "Number=Sing|Person=1|Poss=Yes|PronType=Prs|fPOS=DET++",
                "FORM": "mi",
                "ID": "1",
                "UPOSTAG": "DET"
            }
        }
    }
}
```

In [5]:
# Let Concha do some syntactical analysis with your own phrases
post(
    'http://localhost:5000/v1/documents:analyzeSyntax',
    {"text": "mi mamá me mima"}
)

status=200
{
  "ROOT": {
    "FEATS": "Mood=Ind|Number=Sing|Person=1|Tense=Past|VerbForm=Fin|fPOS=VERB++", 
    "FORM": "mima", 
    "ID": "4", 
    "UPOSTAG": "VERB", 
    "iobj": {
      "FEATS": "Case=Acc,Dat|Number=Sing|Person=1|PrepCase=Npr|PronType=Prs|Reflex=Yes|fPOS=PRON++", 
      "FORM": "me", 
      "ID": "3", 
      "UPOSTAG": "PRON"
    }, 
    "nsubj": {
      "FEATS": "Gender=Fem|Number=Sing|fPOS=NOUN++", 
      "FORM": "mam\u00e1", 
      "ID": "2", 
      "UPOSTAG": "NOUN", 
      "det": {
        "FEATS": "Number=Sing|Person=1|Poss=Yes|PronType=Prs|fPOS=DET++", 
        "FORM": "mi", 
        "ID": "1", 
        "UPOSTAG": "DET"
      }
    }
  }
}



* **POST /v1/documents** It is used as the way to _tell_ Concha new documents to deal with, and recover the details of the associated actions in the answer.

_Request body:_ The text you want Concha deal with
```javascript
{
	"text": "repite hola mundo"
}
```

_Response body:_ It holds the _answer_text_ (the very Concha's answer to the given document), a sequential _id_ numbering the documents as they arrive, the syntactical analysis of the document _request_ (equal to the documents:analyzeSyntax response) and a list of the applied _tricks_ in the order they where used.
```javascript
{
    "answer_text": "hola mundo",
    "id": 3,
    "request": {
        "ROOT": {
            "FEATS": "Mood=Sub|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|fPOS=VERB++",
            "FORM": "repite",
            "ID": "1",
            "UPOSTAG": "VERB",
            "dobj": {
                "FEATS": "Gender=Masc|Number=Sing|fPOS=NOUN++",
                "FORM": "mundo",
                "ID": "3",
                "UPOSTAG": "NOUN",
                "amod": {
                    "FEATS": "Number=Sing|fPOS=ADJ++",
                    "FORM": "hola",
                    "ID": "2",
                    "UPOSTAG": "ADJ"
                }
            }
        }
    },
    "tricks": [
        0
    ]
}```

It has not been explained yet a clue about how to deal with _tricks_, so any usage up to now should fail returning a Concha proprietary error status code (600).

In [6]:
# There should yield a failure sending a document 
# when Concha hasn learnt no trick yet:
"""
status=600
{
  "answer_text": "{'FORM': 'NoTrick'}", 
  "id": ......
"""
post(
    'http://localhost:5000/v1/documents',
    {"text": "repite hola mundo"}
)

status=600
{
  "answer_text": "{'FORM': 'NoTrick'}", 
  "id": 0, 
  "request": {
    "ROOT": {
      "FEATS": "Mood=Sub|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|fPOS=VERB++", 
      "FORM": "repite", 
      "ID": "1", 
      "UPOSTAG": "VERB", 
      "dobj": {
        "FEATS": "Gender=Masc|Number=Sing|fPOS=NOUN++", 
        "FORM": "mundo", 
        "ID": "3", 
        "UPOSTAG": "NOUN", 
        "amod": {
          "FEATS": "Number=Sing|fPOS=ADJ++", 
          "FORM": "hola", 
          "ID": "2", 
          "UPOSTAG": "ADJ"
        }
      }
    }
  }, 
  "tricks": []
}



* **GET /v1/documents** It returns the list of previously received and processed documents to date, with an associated timestamp.

_Response body:_ A list with the _date_ timestamp and the _text_ of each of the previously received documents.
```javascript
[
    {
        "date": "2018-02-06 21:04:15",
        "text": "repite hola mundo"
    },
    {
        "date": "2018-02-06 23:09:21",
        "text": "repite lo más difícil que te salga"
    }
]
```

In [7]:
# Documents list should hold all the 
# (failed) documents you have sent up to now:
get('http://localhost:5000/v1/documents')

status=200
[
  {
    "date": "2018-02-09 19:08:48", 
    "text": "repite hola mundo"
  }
]



## Tricks
Tricks resources are stored in memory (no persistence yet) in a sequential way (append only).

### Trick structure
Every trick has an _antecedent_ (`given`) and a _consequent_ (`then`) part, and some (most of them) have an optional _action_ (`when`) part. Any trick will be triggeed if the current document (or its derivatives) matches the `given`. In that case, if there's a `when` part it will be executed. Lastly, the `then` part will generate a final or intermediate answer with the combination of some new words, part of the incoming document and/or part of the action response. Tricks will be recursively self-assembled to get to the appropriate answer.
```javascript
{
	"given": {
		"ROOT": {
			"FORM": "da",
			"dobj": {
				"FORM": "*algo"
			},
			"iobj": {
				"FORM": "*alguien",
				"case": {
					"FORM": "a"
				}
			}
		}
	},
	"then": {
		"200": "Le voy a dar {d[ROOT][dobj]} , si no hay inconveniente , {d[ROOT][iobj]}",
		"400": "No puedo darlo, no me has especificado qué dar o a quién"
	}
}
```
**Given part** holds a subset of a Universal Dependencies syntactic analysis, holding the most important parts of your favourite phrase you want to react to. Usually you will focus on _FORM_ fields (the very words) and the syntactical dependencies (keys of the object tree). The values of the _FORM_ fields have a different meaning according to its first letter:
* **No special starting character:** The word must match as is.
* **Word starting by ~ character:** It means it could match with a similar word (not implemented yet, doing right now an "as is" matching).
* **Word starting by * character:** It means it will match with any word (remaining word acts as a explainatory comment).

**When part** will be explained later as it is used. Tricks with no `when` part are actionless tricks who act just as word mappers.

** Then part** will generate a combined text response according to the status code of the action, if any (actionless tricks are assumed to always return `200`).

### Trick sub-text selection format
In order to get a part of a text, Concha uses the `python str.format()` syntax, nesting `object[child][grandchild]...`. Those references will be replaced for the full branch the references points to (i.e. the full direct object, as much words it is built of). When accessing part of the incoming _document_ (to the trick, it could be an intermediate incoming document), references will start by letter "d", as you can see in the previous example. When accesing the action _response_ they will start by letter "r", followed by `[body]` (it would access in a future to header content).

### Tricks Methods
Methods are as few as possible.
* **POST /v1/tricks** Sets up a new trick.

_Request body:_ The trick as stated in the trick structure explanation.

_Response body:_ A JSON indicating the sequential _id_ of the new trick and a message with the final status (HTTP status code indicates if there's some issue). It uses to return a 201 code.

In [8]:
# Let's teach Concha its first new trick:
post(
    'http://localhost:5000/v1/tricks',
    {
        "given": {
            "ROOT": {
                "FORM": "repite",
                "dobj": {
                    "FORM": "*algo"
                }
            }
        },
        "then": {
            "200": "{d[ROOT][dobj]}",
            "400": "No puedo repetirlo, no me has especificado qué repetir"
        }
    }
)

status=201
{
  "id": 0, 
  "message": "trick 0 created Ok"
}



In [9]:
# Let make Concha do the first "hello world"
post(
    'http://localhost:5000/v1/documents',
    {"text": "repite hola mundo"}
)

status=201
{
  "answer_text": "hola mundo", 
  "id": 1, 
  "request": {
    "ROOT": {
      "FEATS": "Mood=Sub|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|fPOS=VERB++", 
      "FORM": "repite", 
      "ID": "1", 
      "UPOSTAG": "VERB", 
      "dobj": {
        "FEATS": "Gender=Masc|Number=Sing|fPOS=NOUN++", 
        "FORM": "mundo", 
        "ID": "3", 
        "UPOSTAG": "NOUN", 
        "amod": {
          "FEATS": "Number=Sing|fPOS=ADJ++", 
          "FORM": "hola", 
          "ID": "2", 
          "UPOSTAG": "ADJ"
        }
      }
    }
  }, 
  "tricks": [
    0
  ]
}



* **GET /v1/tricks** The list of all tricks with its internals.
* **GET /v1/tricks/trick_id** The internals of a given trick (if it already exists).
* **PUT /v1/tricks/trick_id** Modify the internals of a given trick (if it already exists).
* **DELETE /v1/tricks/trick_id** Delete a given trick (if it already exists).

In [10]:
# Another example of multiple sub-texting trick:
post(
    'http://localhost:5000/v1/tricks',
    {
        "given": {
            "ROOT": {
                "FORM": "~da",
                "dobj": {
                    "FORM": "*algo"
                },
                "iobj": {
                    "FORM": "*alguien",
                    "case": {
                        "FORM": "a"
                    }
                }
            }
        },
        "then": {
            "200": "Le voy a dar {d[ROOT][dobj]} , si no hay inconveniente , {d[ROOT][iobj]}",
            "400": "No puedo darlo, no me has especificado qué dar o a quién"
        }
    }
)

status=201
{
  "id": 1, 
  "message": "trick 1 created Ok"
}



In [11]:
# Now we can see there are several operative tricks
get('http://localhost:5000/v1/tricks')

status=200
[
  {
    "given": {
      "ROOT": {
        "FORM": "repite", 
        "dobj": {
          "FORM": "*algo"
        }
      }
    }, 
    "then": {
      "200": "{d[ROOT][dobj]}", 
      "400": "No puedo repetirlo, no me has especificado qu\u00e9 repetir"
    }
  }, 
  {
    "given": {
      "ROOT": {
        "FORM": "~da", 
        "dobj": {
          "FORM": "*algo"
        }, 
        "iobj": {
          "FORM": "*alguien", 
          "case": {
            "FORM": "a"
          }
        }
      }
    }, 
    "then": {
      "200": "Le voy a dar {d[ROOT][dobj]} , si no hay inconveniente , {d[ROOT][iobj]}", 
      "400": "No puedo darlo, no me has especificado qu\u00e9 dar o a qui\u00e9n"
    }
  }
]



## GET and POST Tricks
Besides actionless tricks, first operative tricks are the GET Tricks and the POST Tricks, which involve HTTP GET and POST requests to third parties. The request definition is exposed in the `when` part (mandatory in this case). There are no change in the tricks methods to handle GET and POST Tricks, due they are are valid to all kind of tricks.

### GET usage
The `when` part declares two fields:
* The **method** you are using, in this case restricted to GET.
* The **uri** where the very HTTP request is declared along with document sub-text selection completion elements (i.e. `http://server.name:port/whatever/url/{d[some][document][path][FORM]}`). 

Notice the _uri_ sub-text should start from "d" due there's just a _document_ input to deal with. Notice as well it uses to end up the document path with a `[FORM]` node, which means the very word which point to should be used to replace the curly brackets. Replacement sub-text can appear more than once.

Once the `when` part has been prepared for execution (after the trick has been matched), the HTTP uri (with the sub-text replacements) is called with the desired method in a **synchronous** way. The response value is stored in another object named `r` accesible through sub-texting as well in the next to start `then` part.

In [12]:
# Let's define a third trick, this time a GET one:
post(
    'http://localhost:5000/v1/tricks',
    {
        "given": {
            "ROOT": {
                "FORM": "~carga",
                "nmod": {
                    "FORM": "~servidor",
                    "case": {
                        "FORM": "de"
                    },
                    "appos": {
                        "FORM": "*nombre"
                    }
                }
            }
        },
        "when": {
            "method": "GET",
            "uri": "http://127.0.0.1:6000/v1/servers/{d[ROOT][nmod][appos][FORM]}"
        },
        "then": {
            "200": "La carga de {d[ROOT][nmod][appos][FORM]} es de {r[body][load]}",
            "400": "no puedo saberla, no me has dado suficiente información"
        }
    }
)

# New trick can be automatically matched with new document requests
post(
    'http://localhost:5000/v1/documents',
    {"text": "la carga de el servidor srv1"}
)

status=201
{
  "id": 2, 
  "message": "trick 2 created Ok"
}

status=201
{
  "answer_text": "La carga de srv1 es de 10", 
  "id": 2, 
  "request": {
    "ROOT": {
      "FEATS": "Gender=Fem|Number=Sing|fPOS=NOUN++", 
      "FORM": "carga", 
      "ID": "2", 
      "UPOSTAG": "NOUN", 
      "det": {
        "FEATS": "Definite=Def|Gender=Fem|Number=Sing|PronType=Art|fPOS=DET++", 
        "FORM": "la", 
        "ID": "1", 
        "UPOSTAG": "DET"
      }, 
      "nmod": {
        "FEATS": "Gender=Masc|Number=Sing|fPOS=NOUN++", 
        "FORM": "servidor", 
        "ID": "5", 
        "UPOSTAG": "NOUN", 
        "appos": {
          "FEATS": "fPOS=X++", 
          "FORM": "srv1", 
          "ID": "6", 
          "UPOSTAG": "NUM"
        }, 
        "case": {
          "FEATS": "fPOS=ADP++", 
          "FORM": "de", 
          "ID": "3", 
          "UPOSTAG": "ADP"
        }, 
        "det": {
          "FEATS": "Definite=Def|Gender=Masc|Number=Sing|PronType=Art|fPOS=DET++", 
          "FO

### POST usage
The `when` part declares three fields:
* The **method** you are using, in this case restricted to POST.
* The **uri** where the very HTTP request is declared along with document sub-text selection completion elements (i.e. `http://server.name:port/whatever/url/{d[some][document][path][FORM]}`).
* The **body** you want to send along with the POST message to detail how to perform the action.

The sub-text selection can be applied in a similar way as the GET Tricks, but this time it can be applied as many times as you want to the _body_ as well.

Execution and response are similar to GET Tricks.

In [13]:
# Let's define a fourth trick, this time a POST one:
post(
    'http://localhost:5000/v1/tricks',
    {
        "given": {
            "ROOT": {
                "FORM": "~arranca",
                "dobj": {
                    "FORM": "~servidor",
                    "nummod": {
                        "FORM": "*nombre"
                    }
                }
            }
        },
        "when": {
            "method": "POST",
            "uri": "http://127.0.0.1:6000/v1/servers/{d[ROOT][dobj][nummod][FORM]}",
            "body": {
                "kind": "cheap machine"
            }
        },
        "then": {
            "201": "Servidor {d[ROOT][dobj][nummod]} arrancado",
            "400": "No puedo arrancarlo, algo pasa",
            "409": "El servidor {d[ROOT][dobj][nummod]} ya estaba arrancado"
        }
    }
)

# It should fail asking to boot a booted server
post(
    'http://localhost:5000/v1/documents',
    {"text": "arranca el servidor srv1"}
)

# It should work asking to boot a new server
post(
    'http://localhost:5000/v1/documents',
    {"text": "arranca el servidor srv7"}
)

status=201
{
  "id": 3, 
  "message": "trick 3 created Ok"
}

status=409
{
  "answer_text": "El servidor srv1 ya estaba arrancado", 
  "id": 3, 
  "request": {
    "ROOT": {
      "FEATS": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|fPOS=VERB++", 
      "FORM": "arranca", 
      "ID": "1", 
      "UPOSTAG": "VERB", 
      "dobj": {
        "FEATS": "Gender=Masc|Number=Sing|fPOS=NOUN++", 
        "FORM": "servidor", 
        "ID": "3", 
        "UPOSTAG": "NOUN", 
        "det": {
          "FEATS": "Definite=Def|Gender=Masc|Number=Sing|PronType=Art|fPOS=DET++", 
          "FORM": "el", 
          "ID": "2", 
          "UPOSTAG": "DET"
        }, 
        "nummod": {
          "FEATS": "NumType=Card|fPOS=NUM++", 
          "FORM": "srv1", 
          "ID": "4", 
          "UPOSTAG": "NUM"
        }
      }
    }
  }, 
  "tricks": [
    3
  ]
}

status=409
{
  "answer_text": "El servidor srv7 ya estaba arrancado", 
  "id": 4, 
  "request": {
    "ROOT": {
      "FEATS": "Mood=I

## TREAT Tricks and dependent clauses
The most interesting tricks are the TREAT Tricks because they can help in the Tricks self-assembly to solve multi clause compound phrases. This doesn't involve HTTP GET and POST requests to third parties but the internal treatment of phrase sub-text clauses, processing them with other Tricks and finally replacing the result in the original document. 

The real power of this kind of Trick is it can be automatically recursive. Concha will try all the combinations and **current selection chriteria** will take the longest treatment (more Tricks recursively involved) and if there's a tie the lower status code and, finally, if there's a tie, a random one.

And yes, TREAT Trick names were chosen because the _"trick or treat"_ fun! :-D

### TREAT usage
The `when` part declares two fields:
* The **method** you are using, in this case restricted to TREAT.
* The **uri** which is declared just with document one sub-text selection (i.e. `{d[some][document][path]}`). 

Once the `when` part has been prepared for execution (after the trick has been matched), the sub-text replacementsis re-parsed as a new document and treated recursively among all tricks in a **synchronous** way. The selected response value, according to the selection chriteria, is stored in another object named `r` accesible through sub-texting as well in the next to start `then` part.

The `then` part is a little bit tricky, because it doesn't declare the result of the Trick, but the result of the replacement, which can be a combination of extra words, part or all of the response value and part of the original document. The result of the Trick will be the rest of the document who wasn't treated, as is, **and** the tesult of the `then` declaration as the substitution of the _uri_ sub-text. 

**NOTE:** the selected sub-text response can be acting isolated in a different syntactical manner than it was acting in the incoming document _before_ the replacement, and in a different syntactical manner than it will act in the Trick outcome result _after_ the replacement. Because of that there are several internal re-parsings in the process.

In [14]:
# Let's define a fifth trick, this time a TREAT one.
# It declares it is interesting to dig into the
# direct object as a possible dependent clause.
post(
    'http://localhost:5000/v1/tricks',
    {
        "given": {
            "ROOT": {
                "FORM": "*haz",
                "dobj": {
                    "FORM": "*algo"
                }
            }
        },
        "when": {
            "method": "TREAT",
            "uri": "{d[ROOT][dobj]}"
        },
        "then": {
            "200": "{r[ROOT]}"
        }
    }
)

status=201
{
  "id": 4, 
  "message": "trick 4 created Ok"
}



### Mixing up it all

Let's make a basic set-up in the existing servers srv1 and srv7. Remember each process consumes 10 units in each server.

In [15]:
# Deploy some processes on existing servers srv1 and srv7
post(
    'http://localhost:6000/v1/servers/srv1/processes/p1',
    {"kind": "tetris"}
)
post(
    'http://localhost:6000/v1/servers/srv7/processes/p1',
    {"kind": "pacman"}
)
post(
    'http://localhost:6000/v1/servers/srv7/processes/p2',
    {"kind": "pong"}
)
# Check one have more load than other
get('http://localhost:6000/v1/servers')

status=201
{
  "message": "process p1 launched Ok in server srv1"
}

status=201
{
  "message": "process p1 launched Ok in server srv7"
}

status=201
{
  "message": "process p2 launched Ok in server srv7"
}

status=200
{
  "srv1": {
    "description": {
      "kind": "cheap machine"
    }, 
    "load": 20, 
    "name": "srv1", 
    "p1": {
      "description": {
        "kind": "tetris"
      }, 
      "name": "p1"
    }, 
    "processes": {}
  }, 
  "srv7": {
    "description": {
      "kind": "cheap machine"
    }, 
    "load": 40, 
    "name": "srv7", 
    "p1": {
      "description": {
        "kind": "pacman"
      }, 
      "name": "p1"
    }, 
    "p2": {
      "description": {
        "kind": "pong"
      }, 
      "name": "p2"
    }, 
    "processes": {}
  }
}



Now we're going to manage a useful compound phrase: We are running out of budget and we want Concha to shut down the server who is consuming the biggest processes load.

In orther to do that, we need to teach Concha a couple of new tricks: 

In [16]:
# Identify the name of the most consuming server.
post(
    'http://localhost:5000/v1/tricks',
    {
        "given": {
            "ROOT": {
                "FORM": "servidor",
                "acl:relcl": {
                    "FORM": "gasta",
                    "advmod": {
                        "FORM": "más"
                    },
                    "mark": {
                        "FORM": "que"
                    }
                },
                "det": {
                    "FORM": "el"
                }
            }
        },
        "when": {
            "method": "GET",
            "uri": "http://127.0.0.1:6000/v1/servers?filter=max_load"
        },
        "then": {
            "200": "el servidor {r[body][name]}"
        }
    }
)

status=201
{
  "id": 5, 
  "message": "trick 5 created Ok"
}



In [17]:
# Shut down a named server
post(
    'http://localhost:5000/v1/tricks',
    {
        "given": {
            "ROOT": {
                "FORM": "~apaga",
                "dobj": {
                    "FORM": "~servidor",
                    "appos": {
                        "FORM": "*nombre"
                    }
                }
            }
        },
        "when": {
            "method": "DELETE",
            "uri": "http://localhost:6000/v1/servers/{d[ROOT][dobj][appos][FORM]}"
        },
        "then": {
            "200": "Servidor {d[ROOT][dobj][appos]} apagado",
            "400": "No puedo apagarlo, algo pasa",
            "404": "El servidor {d[ROOT][dobj][appos]} ya estaba apagado"
        }
    }
)

status=201
{
  "id": 6, 
  "message": "trick 6 created Ok"
}



At last, we can ask Concha to react to a compound phrase.

In [18]:
# Locate the server with higest load and shut it down.
post(
    'http://localhost:5000/v1/documents',
    {"text": "apaga el servidor que más gasta"}
)

status=201
{
  "answer_text": "Servidor srv7 apagado", 
  "id": 5, 
  "request": {
    "ROOT": {
      "FEATS": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|fPOS=VERB++", 
      "FORM": "apaga", 
      "ID": "1", 
      "UPOSTAG": "VERB", 
      "dobj": {
        "FEATS": "Gender=Masc|Number=Sing|fPOS=NOUN++", 
        "FORM": "servidor", 
        "ID": "3", 
        "UPOSTAG": "NOUN", 
        "acl:relcl": {
          "FEATS": "Number=Sing|fPOS=ADJ++", 
          "FORM": "gasta", 
          "ID": "6", 
          "UPOSTAG": "ADJ", 
          "advmod": {
            "FEATS": "Degree=Cmp|fPOS=ADV++", 
            "FORM": "m\u00e1s", 
            "ID": "5", 
            "UPOSTAG": "ADV"
          }, 
          "mark": {
            "FEATS": "fPOS=SCONJ++", 
            "FORM": "que", 
            "ID": "4", 
            "UPOSTAG": "SCONJ"
          }
        }, 
        "det": {
          "FEATS": "Definite=Def|Gender=Masc|Number=Sing|PronType=Art|fPOS=DET++", 
          "FORM":

### Well done!!!
You have completed a full compound phrase analyzing part of it as different clauses and then the result has been analyzed again till a result has been achieved.

What happened? 

It was applied a lot of tricks, but the trick sequence which ended up in the final `answer_text` (_"Servidor srv7 apagado"_) is the one marked in the aswer field `"tricks"` (this is [4, 5, 6]). You can check in the upper cells that there were those ones:

* 4 = "ROOT": {"FORM": "*haz", "dobj": {"FORM": "*algo"}}
* 5 = "ROOT": {"FORM": "servidor", "acl:relcl": {"FORM": "gasta", "advmod": {"FORM": "más"}, "mark": {"FORM": "que"}}
* 6 = "ROOT": {"FORM": "~apaga", "dobj": {"FORM": "~servidor", "appos": {"FORM": "*nombre"}}}

There was a replacement of the `dobj` of the original phrase ("el servidor que más gasta") by the result of the result of the finding of the trick 5 (which yield "el servidor srv7") and finally there was a call to the trick 6 (doing an "apaga el servidor srv7") getting its result as the final result.

Now you can teach Concha your own tricks. Enjoy!